In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,314 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [0]:
#Extracting text from images using tessearct

import os.path
import json
import io
import sys
import string
import pytesseract
import re
import difflib
import csv
import dateutil.parser as dparser
try:
	from PIL import Image, ImageEnhance, ImageFilter
except:
	print("Please Install PIL - For Python 3 Users the Library is now called Pillow")
	sys.exit()
path = sys.argv[1]

In [4]:
from google.colab import files
uploaded = files.upload()

Saving aadhar_or_pan_2.jpeg to aadhar_or_pan_2.jpeg


In [0]:
img = Image.open('aadhar_or_pan_2.jpeg')
img = img.convert('RGB')
pix = img.load()

In [20]:
for y in range(img.size[1]):
  for x in range(img.size[0]):
    if pix[x, y][0] < 102 or pix[x, y][1] < 102 or pix[x, y][2] < 102:
      pix[x, y] = (0, 0, 0, 255)
    else:
      pix[x, y] = (255, 255, 255, 255)

img.save('temp.jpg')

text_in = pytesseract.image_to_string(Image.open('temp.jpg'))
text = list(filter(lambda x: ord(x)<128, text_in))  # TO BE CHECKED
print(text_in)

text_output = open('outputbase.txt', 'w')
text_output.write(text_in)
text_output.close()

file = open('outputbase.txt', 'r')
text = file.read()
print(text)

name = None
fname = None
dob = None
pan = None
nameline = []
dobline = []
panline = []
text0 = []
text1 = []
text2 = []

# Searching for PAN
lines = text.split('\n')
for lin in lines:
  s = lin.strip()
  s = s.rstrip()
  s = s.lstrip('_')
  text1.append(s)

text1 = list(filter(None, text1))
print(text1)

lineno=0 # to start from the first line of the text file.

for wordline in text1:
  xx = wordline.split('\n')
  if ([w for w in xx if re.search('(INCOME|TAX|GOW|GOVT|GOVERNMENT|OVERNMENT|VERNMENT|DEPARTMENT|EPARTMENT|PARTMENT|ARTMENT|INDIA|NDIA)$', w)]):
    text1 = list(text1)
    lineno = text1.index(wordline)
    break
print(text1)

text0 = text1[lineno+1:]
print(text0)

def findword(textlist, wordstring):
  lineno = -1
  for wordline in textlist:
      xx = wordline.split( )
      if ([w for w in xx if re.search(wordstring, w)]):
          lineno = textlist.index(wordline)
          textlist = textlist[lineno+1:]
          return textlist
  return textlist

try:

    # Cleaning first names, better accuracy
    name = text0[0]
    name = name.rstrip()
    name = name.lstrip()
    name = name.replace("8", "B")
    name = name.replace("0", "D")
    name = name.replace("6", "G")
    name = name.replace("1", "I")
    name = re.sub('[^a-zA-Z] +', ' ', name)

    # Cleaning Father's name
    fname = text0[1]
    fname = fname.rstrip()
    fname = fname.lstrip()
    fname = fname.replace("8", "S")
    fname = fname.replace("0", "O")
    fname = fname.replace("6", "G")
    fname = fname.replace("1", "I")
    fname = fname.replace("\"","A")
    fname = re.sub('[^a-zA-Z] +', ' ', fname)

    # Cleaning DOB
    dob = text0[2]
    dob = dob.rstrip()
    dob = dob.lstrip()
    dob = dob.replace('l', '/')
    dob = dob.replace('L', '/')
    dob = dob.replace('I', '/')
    dob = dob.replace('i', '/')
    dob = dob.replace('|', '/')
    dob = dob.replace('\"', '/1')
    dob = dob.replace(" ", "")

    # Cleaning PAN Card details
    text0 = findword(text1, '(Pormanam|Number|umber|Account|ccount|count|Permanent|ermanent|manent|wumm)$')
    panline = text0[0]
    pan = panline.rstrip()
    pan = pan.lstrip()
    pan = pan.replace(" ", "")
    pan = pan.replace("\"", "")
    pan = pan.replace(";",  "")
    pan = pan.replace("%", "L")

except:
    pass    

data = {}
data['Name'] = name
data['Father Name'] = fname
data['Date of Birth'] = dob
data['PAN'] = pan

print(data)

try:
  to_unicode = unicode
except NameError:
  to_unicode = str

with io.open('data.json', 'w', encoding='utf8') as outfile:
  str_ = json.dumps(data,
          indent=4, sort_keys=True,
          separators=(',', ': '), ensure_ascii=False)
  outfile.write(to_unicode(str_))

with open('data.json') as data_file:
  data_loaded = json.load(data_file)

print(data == data_loaded)

# Reading data back JSON(give correct path where JSON is stored)
with open('data.json', 'r') as f:
  ndata = json.load(f)

print('\t', "|+++++++++++++++++++++++++++++++|")
print('\t', '|', '\t', ndata['Name'])
print('\t', "|-------------------------------|")
print('\t', '|', '\t', ndata['Father Name'])
print('\t', "|-------------------------------|")
print('\t', '|', '\t', ndata['Date of Birth'])
print('\t', "|-------------------------------|")
print('\t', '|', '\t', ndata['PAN'])
print('\t', "|+++++++++++++++++++++++++++++++|")

seer faart RRA Wea

INCOME TAX DEPARTMENT 3 GOVT. OF INDIA
D MANIKANDAN ” PG

   

DURAISAMY
16/07/1986,

Pernument Aucaunt Number

BNZPM2501F

Deronda oben

Signature
seer faart RRA Wea

INCOME TAX DEPARTMENT 3 GOVT. OF INDIA
D MANIKANDAN ” PG

   

DURAISAMY
16/07/1986,

Pernument Aucaunt Number

BNZPM2501F

Deronda oben

Signature
['seer faart RRA Wea', 'INCOME TAX DEPARTMENT 3 GOVT. OF INDIA', 'D MANIKANDAN ” PG', 'DURAISAMY', '16/07/1986,', 'Pernument Aucaunt Number', 'BNZPM2501F', 'Deronda oben', 'Signature']
['seer faart RRA Wea', 'INCOME TAX DEPARTMENT 3 GOVT. OF INDIA', 'D MANIKANDAN ” PG', 'DURAISAMY', '16/07/1986,', 'Pernument Aucaunt Number', 'BNZPM2501F', 'Deronda oben', 'Signature']
['D MANIKANDAN ” PG', 'DURAISAMY', '16/07/1986,', 'Pernument Aucaunt Number', 'BNZPM2501F', 'Deronda oben', 'Signature']
{'Name': 'D MANIKANDAN  PG', 'Father Name': 'DURAISAMY', 'Date of Birth': '16/07/1986,', 'PAN': 'BNZPM2501F'}
True
	 |+++++++++++++++++++++++++++++++|
	 | 	 D MANIKANDAN  PG